In [67]:
pip install yfinance pandas numpy scipy


Note: you may need to restart the kernel to use updated packages.


In [68]:
import yfinance as yf
import pandas as pd

ticker = 'AAPL'
stock = yf.Ticker(ticker)
expiration_dates = stock.options

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(stock)

yfinance.Ticker object <AAPL>


In [69]:
option_data = []

for expiration in expiration_dates:
    options = stock.option_chain(expiration)
    options.calls['expirationDate'] = expiration
    options.puts['expirationDate'] = expiration
    option_data.append(options.calls)
    option_data.append(options.puts)

# Combine all options data into a single DataFrame
option_chain = pd.concat(option_data, ignore_index=True)

Importing the Option chain data for apple stocks

In [70]:
option_chain.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expirationDate
0,AAPL240524C00100000,2024-05-20 14:15:46+00:00,100.0,91.49,0.00,0.00,0.0,0.0,1.0,8,0.00001,True,REGULAR,USD,2024-05-24
1,AAPL240524C00105000,2024-05-14 19:54:01+00:00,105.0,82.30,0.00,0.00,0.0,0.0,1.0,1,0.00001,True,REGULAR,USD,2024-05-24
2,AAPL240524C00115000,2024-04-29 17:14:13+00:00,115.0,59.90,0.00,0.00,0.0,0.0,NaN,0,0.00001,True,REGULAR,USD,2024-05-24
3,AAPL240524C00120000,2024-04-12 14:24:01+00:00,120.0,57.80,62.85,63.85,0.0,0.0,1.0,1,0.00001,True,REGULAR,USD,2024-05-24
4,AAPL240524C00125000,2024-05-20 14:10:59+00:00,125.0,66.34,0.00,0.00,0.0,0.0,1.0,3,0.00001,True,REGULAR,USD,2024-05-24


In [71]:
current_price = stock.history(period='1d')['Close'].iloc[0]

In [72]:
option_chain.shape

(1782, 15)

Calculation of ITM/OTM/ATM and appending it as a new column on imported data

In [73]:
option_chain['ITM/OTM/ATM'] = ''

In [74]:
for index, row in option_chain.iterrows():
    if row['contractSymbol'][10] == 'C':
        if row['lastPrice'] > row['strike']:
            option_chain.at[index, 'ITM/OTM/ATM'] = 'ITM'
        elif row['lastPrice'] < row['strike']:
            option_chain.at[index, 'ITM/OTM/ATM'] = 'OTM'
        else:
            option_chain.at[index, 'ITM/OTM/ATM'] = 'ATM'
    else:
        if row['lastPrice'] > row['strike']:
            option_chain.at[index, 'ITM/OTM/ATM'] = 'OTM'
        elif row['lastPrice'] < row['strike']:
            option_chain.at[index, 'ITM/OTM/ATM'] = 'ITM'
        else:
            option_chain.at[index, 'ITM/OTM/ATM'] = 'ATM'

In [75]:
option_chain.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expirationDate,ITM/OTM/ATM
0,AAPL240524C00100000,2024-05-20 14:15:46+00:00,100.0,91.49,0.00,0.00,0.0,0.0,1.0,8,0.00001,True,REGULAR,USD,2024-05-24,OTM
1,AAPL240524C00105000,2024-05-14 19:54:01+00:00,105.0,82.30,0.00,0.00,0.0,0.0,1.0,1,0.00001,True,REGULAR,USD,2024-05-24,OTM
2,AAPL240524C00115000,2024-04-29 17:14:13+00:00,115.0,59.90,0.00,0.00,0.0,0.0,NaN,0,0.00001,True,REGULAR,USD,2024-05-24,OTM
3,AAPL240524C00120000,2024-04-12 14:24:01+00:00,120.0,57.80,62.85,63.85,0.0,0.0,1.0,1,0.00001,True,REGULAR,USD,2024-05-24,OTM
4,AAPL240524C00125000,2024-05-20 14:10:59+00:00,125.0,66.34,0.00,0.00,0.0,0.0,1.0,3,0.00001,True,REGULAR,USD,2024-05-24,OTM


Calculate the fair values of these options using Black-Scholes-Merton (BSM) and determined whether they are overpriced or underpricedby comparing fair values and closing price..

In [76]:
from scipy.stats import norm
from datetime import datetime

def bsm_option_price(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Option type must be 'call' or 'put'.")
    
    return option_price

In [77]:
risk_free_rate = 0.05

In [78]:
import numpy as np
option_chain['fair_value'] = np.nan
option_chain['over_under_price'] = np.nan

In [79]:
option_chain['lastTradeDate'] = pd.to_datetime(option_chain['lastTradeDate']).dt.tz_localize(None)
option_chain['expirationDate'] = pd.to_datetime(option_chain['expirationDate']).dt.tz_localize(None)

In [80]:
for index, row in option_chain.iterrows():
    S = 100 
    K = row['strike']  # Strike price
    T = (row['expirationDate'] - row['lastTradeDate']).days / 365  # Time to expiration in years
    sigma = row['impliedVolatility']  # Implied volatility
    
    option_type = 'call' if row['contractSymbol'][10] == 'C' else 'put'
    
    fair_value = bsm_option_price(S, K, T, risk_free_rate, sigma, option_type)
    option_chain.at[index, 'fair_value'] = fair_value
    
    if row['lastPrice'] > fair_value:
        option_chain.at[index, 'over_under_price'] = 'Overpriced'
    elif row['lastPrice'] < fair_value:
        option_chain.at[index, 'over_under_price'] = 'Underpriced'
    else:
        option_chain.at[index, 'over_under_price'] = 'Fairly Priced'

C:\Users\kunal\AppData\Local\Temp\ipykernel_16160\3195169063.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Overpriced' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  option_chain.at[index, 'over_under_price'] = 'Overpriced'


In [81]:
option_chain

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expirationDate,ITM/OTM/ATM,fair_value,over_under_price
0,AAPL240524C00100000,2024-05-20 14:15:46,100.0,91.49,0.00,0.00,0.0,0.0,1.0,8,0.000010,True,REGULAR,USD,2024-05-24,OTM,4.108745e-02,Overpriced
1,AAPL240524C00105000,2024-05-14 19:54:01,105.0,82.30,0.00,0.00,0.0,0.0,1.0,1,0.000010,True,REGULAR,USD,2024-05-24,OTM,0.000000e+00,Overpriced
2,AAPL240524C00115000,2024-04-29 17:14:13,115.0,59.90,0.00,0.00,0.0,0.0,NaN,0,0.000010,True,REGULAR,USD,2024-05-24,OTM,0.000000e+00,Overpriced
3,AAPL240524C00120000,2024-04-12 14:24:01,120.0,57.80,62.85,63.85,0.0,0.0,1.0,1,0.000010,True,REGULAR,USD,2024-05-24,OTM,0.000000e+00,Overpriced
4,AAPL240524C00125000,2024-05-20 14:10:59,125.0,66.34,0.00,0.00,0.0,0.0,1.0,3,0.000010,True,REGULAR,USD,2024-05-24,OTM,0.000000e+00,Overpriced
5,AAPL240524C00130000,2024-05-20 16:52:32,130.0,61.65,0.00,0.00,0.0,0.0,1.0,2,0.000010,True,REGULAR,USD,2024-05-24,OTM,0.000000e+00,Overpriced
6,AAPL240524C00135000,2024-05-14 19:57:05,135.0,52.98,0.00,0.00,0.0,0.0,3.0,4,0.000010,True,REGULAR,USD,2024-05-24,OTM,0.000000e+00,Overpriced
7,AAPL240524C00140000,2024-05-16 14:25:46,140.0,50.80,0.00,0.00,0.0,0.0,2.0,15,0.000010,True,REGULAR,USD,2024-05-24,OTM,0.000000e+00,Overpriced
8,AAPL240524C00145000,2024-05-20 17:59:15,145.0,46.75,0.00,0.00,0.0,0.0,22.0,41,0.000010,True,REGULAR,USD,2024-05-24,OTM,0.000000e+00,Overpriced
9,AAPL240524C00148000,2024-05-15 16:00:09,148.0,42.12,0.00,0.00,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD,2024-05-24,OTM,0.000000e+00,Overpriced
